In [10]:
import numpy as np
import cv2

In [11]:
def load_image(image_path):
    image = cv2.imread(image_path)
    return image

def otsu_limiar(image):
    # CONVERTER PARA ESCALA DE CINZA
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # OTSU
    _, otsu = cv2.threshold(image_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return otsu

def watershed_limiar(image):
    # CONVERTER PARA ESCALA DE CINZA
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # BINARIZAR
    _, binarizada = cv2.threshold(image_gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # ABERTURA PARA REMOVER RUÍDOS
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(binarizada, cv2.MORPH_OPEN, kernel, iterations = 2)
    
    # ENCONTRAR CONTORNOS
    sure_bg = cv2.dilate(opening, kernel, iterations=3)
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(dist_transform, 0.7*dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)
    
    # ROTULAR MARCADORES
    _, markers = cv2.connectedComponents(sure_fg)
    markers += 1
    markers[unknown == 255] = 0
    
    # WATERSHED
    markers = cv2.watershed(image, markers)
    image[markers == -1] = [255, 0, 0]
    
    return image

In [12]:
imagens = ['00', '23', '28', '48', '68', '75', '144']

for i in imagens:
    image = load_image(f"src/org/{i}h.jpg")

    otsu = otsu_limiar(image)
    cv2.imwrite(f'src/lim/otsu/{i}h.jpg', otsu)

    watershed = watershed_limiar(image)
    cv2.imwrite(f'src/lim/watershed/{i}h.jpg', watershed)